In [1]:
from utils.game_map_class import GameMap
from utils.game_team_class import GameTeam
from utils.strategy_class import Strategy, RandomStrategy, RuleOfThumbStrategy, MonteCarloTreeSearchStrategy
from utils.map_setup_functions import setGameBoardRandom, initializeFullRiskMap
from utils.heuristics import BST_Heuristic, EdgeWin, EdgeDiff
from utils.MCTS import monteCarloTreeSearch, getActions

In [2]:
import networkx as nx
import matplotlib.pyplot as plt

In [3]:
import copy
import math
import random
import numpy as np
from collections import defaultdict

In [ ]:
full_risk_map = initializeFullRiskMap()
team_names = ['red', 'blue']
team_strategies = [MonteCarloTreeSearchStrategy, RuleOfThumbStrategy]
risk_map, teams = setGameBoardRandom(team_names, full_risk_map, team_strategies)

In [ ]:
curr_team = teams[0]
defending_team = teams[1]
counter = 0
while defending_team.hasTeamWon() == False:
    curr_team.playTurn()
    curr_team_temp = curr_team
    curr_team = defending_team
    defending_team = curr_team_temp
    color_map = risk_map.getTeamColorMap()
    nx.draw(risk_map.graph, node_color = color_map, with_labels=True)
    plt.show()
    counter += 1
print("{defender} team has won in {num_turns} turns".format(defender = defending_team.getName(), num_turns = counter // 2))

In [4]:

winner_array = []
winner_count = []
starter_array = []
#winner_array = np.load('winners.npy').tolist()

#starter_array = np.load('starters.npy').tolist()
for i in range(100):
    full_risk_map = initializeFullRiskMap()
    if random.choice([0, 1]) == 0:
        team_names = ["blue", "red"]
        team_strategies = [MonteCarloTreeSearchStrategy, RuleOfThumbStrategy]
        starter_array.append("blue")
    else:
        team_names = ["red", "blue"]
        team_strategies = [RuleOfThumbStrategy, MonteCarloTreeSearchStrategy]
        starter_array.append("red")
        
    risk_map, teams = setGameBoardRandom(team_names, full_risk_map, team_strategies)
    #risk_map, teams = setGameBoardRandom(team_names, full_risk_map, RandomStrategy)
    curr_team = teams[0]
    defending_team = teams[1]
    counter = 0
    while defending_team.hasTeamWon() == False:
        curr_team.playTurn()
        curr_team_temp = curr_team
        curr_team = defending_team
        defending_team = curr_team_temp
        color_map = risk_map.getTeamColorMap()
        #nx.draw(risk_map.graph, node_color = color_map, with_labels=True)
        plt.show()
        counter += 1
    
    winner_array.append(defending_team.getName())
    winner_count.append(counter // 2)
    print(i)
    np.save('winners.npy', np.array(winner_array))
    np.save('starters.npy', np.array(starter_array))
    np.save('winners_count.npy', np.array(winner_count))
    print("{defender} team has won in {num_turns} turns".format(defender = defending_team.getName(), num_turns = counter//2))
print(winner_array)
print(winner_count)
print(starter_array)

0
blue team has won in 14 turns
1
blue team has won in 16 turns
2
blue team has won in 10 turns
3
red team has won in 13 turns
4
red team has won in 9 turns
5
red team has won in 15 turns
6
blue team has won in 12 turns
7
blue team has won in 10 turns
8
red team has won in 8 turns
9
blue team has won in 9 turns
10
red team has won in 17 turns
11
blue team has won in 10 turns
12
blue team has won in 9 turns
13
red team has won in 15 turns
14
blue team has won in 14 turns
15
red team has won in 16 turns
16
blue team has won in 9 turns
17
red team has won in 15 turns
18
blue team has won in 10 turns
19
red team has won in 17 turns
20
blue team has won in 15 turns
21
blue team has won in 17 turns
22
red team has won in 11 turns
23
red team has won in 6 turns
24
red team has won in 12 turns
25
red team has won in 6 turns
26
blue team has won in 15 turns
27
blue team has won in 10 turns
28
red team has won in 8 turns
29
blue team has won in 15 turns
30
blue team has won in 12 turns
31
red te

In [5]:
i = 0
for winner in winner_array:
    if winner == 'blue':
        i += 1
print(i)

51
